In [ ]:
!pip install -q sentence-transformers chromadb google-generativeai gtts guardrails-ai

In [ ]:
!pip uninstall -y sympy
!pip install --no-cache-dir sympy transformers

In [ ]:
import json
import chromadb
import numpy as np
import google.generativeai as genai
import IPython.display as ipd
from sentence_transformers import SentenceTransformer
from gtts import gTTS  # using for text-to-speech
import os

In [ ]:
import sqlite3

db_path = "mahindra_university_data.db"

# Connect to SQLite DB
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Fetch all content from the 'pages' table
cursor.execute("SELECT url, title, content FROM pages")
rows = cursor.fetchall()

# Build documents list
documents = []
for row in rows:
    url, title, content = row
    full_text = f"{title}\n{content}\nURL: {url}"
    documents.append(full_text)

print(f"Loaded {len(documents)} documents from SQLite database.")
print("\nSample Document:\n", documents[0])


In [ ]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

embeddings = model.encode(documents, normalize_embeddings=True)

# IChromaDB
chroma_client = chromadb.PersistentClient(path="/content/chroma_db")
collection = chroma_client.get_or_create_collection(name="mahindra_uni")

# Adding documents and embeddings to ChromaDB
for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
    collection.add(
        ids=[str(i)],  # Unique ID for each document
        embeddings=[embedding.tolist()],  # Convert numpy array to list
        documents=[doc]  # Store original document text
    )

print(f"Stored {collection.count()} documents in ChromaDB.")


In [ ]:
def retrieve_top_k(query, k=1):
    query_embedding = model.encode([query], normalize_embeddings=True).tolist()[0]
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k
    )
    return results["documents"][0] if results["documents"] else ["No relevant data found."]

import google.generativeai as genai

genai.configure(api_key="YOUR_API_KEY")

gemini_model = genai.GenerativeModel("gemini-1.5-pro")

def get_gemini_response(query, context):
    prompt = f"""
    You are an AI chatbot assisting students with information about Mahindra University.
    Use the following context to answer the query accurately.

    Context: {context}

    Query: {query}

    Provide a precise and informative answer.
    """
    response = gemini_model.generate_content(prompt)
    return response.text if response else "Sorry, I couldn't generate a response."

#function  (TTS)
def speak(text):
    tts = gTTS(text=text, lang="en")
    tts.save("response.mp3")
    return ipd.Audio("response.mp3", autoplay=True)


In [ ]:
print("Chatbot is running... Type 'exit' to stop.")

while True:
    user_query = input("\nYou: ")

    if user_query.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    retrieved_docs = retrieve_top_k(user_query, k=1)
    chatbot_response = get_gemini_response(user_query, retrieved_docs[0])

    print("\nChatbot:", chatbot_response)

    audio = speak(chatbot_response)
    display(audio)
